Diese Query zieht sich eine Random Zeile aus der Stop Tabelle und printed Attributes

In [1]:
import psycopg2
conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="1234")
with conn.cursor() as cur:
    cur.execute("""
        SELECT xml_member_name, stop_id, eva, ar_ts, dp_ts
        FROM stops
        ORDER BY random()
        LIMIT 1;
    """)
    row = cur.fetchone()

print("PK =", (row[0], row[1]))
print("rest =", {"eva": row[2], "ar_ts": row[3], "dp_ts": row[4]})
conn.close()

PK = ('2509180100/berlin-neuk_lln_timetable.xml', '5920742509040489613-2509180032-16')
rest = {'eva': 8089077, 'ar_ts': datetime.datetime(2025, 9, 18, 1, 7, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))), 'dp_ts': datetime.datetime(2025, 9, 18, 1, 7, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))}


Die Query berechnet die durchschnittliche Verspätung global über ganz Berlin (unter der Bedingung, dass eine Versopätung vorliegt)

In [2]:
import psycopg2
conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="1234")
# Durchschnittliche Abfahrtsverspätung (nur Zeilen mit dp_ts UND actual_departure != NULL)
with conn.cursor() as cur:
    cur.execute("""
        SELECT
            AVG(EXTRACT(EPOCH FROM (actual_departure - dp_ts))) AS avg_delay_seconds,
            COUNT(*) AS n
        FROM public.stops
        WHERE dp_ts IS NOT NULL
          AND actual_departure IS NOT NULL;
    """)
    avg_delay_seconds, n = cur.fetchone()

print("N =", n)
print("Ø Abfahrtsverspätung (Sek.) =", round(avg_delay_seconds, 2))
print("Ø Abfahrtsverspätung (Min.) =", round((float(avg_delay_seconds) / 60.0), 2) if avg_delay_seconds is not None else None)
conn.close()

N = 655526
Ø Abfahrtsverspätung (Sek.) = 105.58
Ø Abfahrtsverspätung (Min.) = 1.76


Diese Query berechnet die durchschnittliche Verspätung bei einer Station (unter der Bedingung, dass eine Verspätung vorliegt) je Station

In [3]:
# Ø Abfahrtsverspätung nach Station (größte zuerst)
import psycopg2
conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="1234")
with conn.cursor() as cur:
    cur.execute("""
        SELECT
            st.eva,
            st.name AS station_name,
            COUNT(*) AS n,
            AVG(EXTRACT(EPOCH FROM (s.actual_departure - s.dp_ts))) AS avg_delay_seconds
        FROM public.stops s
        JOIN public.stationen st
          ON st.eva = s.eva
        WHERE s.dp_ts IS NOT NULL
          AND s.actual_departure IS NOT NULL
        GROUP BY st.eva, st.name
        HAVING COUNT(*) > 0
        ORDER BY avg_delay_seconds DESC;
    """)
    rows = cur.fetchall()

# Beispiel: Top 20 ausgeben
for eva, name, n, avg_s in rows[:20]:
    print(f"{eva} | {name:40s} | n={n:7d} | Ø={avg_s/60:7.2f} min")
conn.close()

8010404 | Berlin-Spandau                           | n=   8569 | Ø=   9.24 min
8011160 | Berlin Hauptbahnhof                      | n=   8499 | Ø=   7.59 min
8011113 | Berlin Südkreuz                          | n=   3787 | Ø=   5.57 min
8010406 | Berlin Zoologischer Garten               | n=   5484 | Ø=   5.26 min
8010405 | Berlin-Wannsee                           | n=   3140 | Ø=   4.62 min
8010403 | Berlin-Charlottenburg                    | n=   2643 | Ø=   4.59 min
8010255 | Berlin Ostbahnhof                        | n=   5790 | Ø=   4.19 min
8011155 | Alexanderplatz                           | n=   4613 | Ø=   3.59 min
8011102 | Berlin Gesundbrunnen                     | n=   4978 | Ø=   3.35 min
8011118 | Berlin Potsdamer Platz                   | n=   3431 | Ø=   3.25 min
8011306 | Berlin-Friedrichstraße                   | n=   5191 | Ø=   3.17 min
8011162 | Berlin Ostkreuz                          | n=   8281 | Ø=   2.92 min
8011041 | Berlin-Lichterfelde Ost                  |

Diese Query gibt die Koordinaten aller Stationen zurück und das folgende Skript visualisiert die Stationen um zu checken, ob die Koordinaten richtig extrhiert wurden.

In [4]:
# Falls nötig (einmalig): !pip install folium
import psycopg2
import folium

# --- DB connect ---
conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="1234")

# 1) Stationen aus DB holen (Berlin-BBox)
BERLIN_BBOX = {
    "lat_min": 52.3,
    "lat_max": 52.7,
    "lon_min": 13.0,
    "lon_max": 13.85,
}

with conn.cursor() as cur:
    cur.execute(
        """
        SELECT eva, name, latitude, longitude
        FROM public.stationen
        WHERE latitude  IS NOT NULL
          AND longitude IS NOT NULL
          AND latitude  BETWEEN %s AND %s
          AND longitude BETWEEN %s AND %s
        """,
        (BERLIN_BBOX["lat_min"], BERLIN_BBOX["lat_max"], BERLIN_BBOX["lon_min"], BERLIN_BBOX["lon_max"]),
    )
    rows = cur.fetchall()

print("Stationen (Berlin-BBox):", len(rows))

# 2) Karten-Zentrum bestimmen
if rows:
    lats = [r[2] for r in rows]
    lons = [r[3] for r in rows]
    center = [sum(lats) / len(lats), sum(lons) / len(lons)]
else:
    center = [52.52, 13.405]

# 3) OSM-Karte erzeugen (kein Clustering, crisp Marker)
POINT_RADIUS = 7
FONT_SIZE_PX = 16

m = folium.Map(
    location=center,
    zoom_start=11,
    tiles="OpenStreetMap",
    prefer_canvas=True,  # bei vielen Punkten oft performanter/cleaner
)

fg = folium.FeatureGroup(name="Stationen", show=True).add_to(m)

for eva, name, lat, lon in rows:
    tooltip_html = f'<div style="font-size:{FONT_SIZE_PX}px; font-weight:600;">{name}</div>'
    popup_html = f"""
    <div style="font-size:{FONT_SIZE_PX}px;">
      <b>{name}</b><br>
      EVA: {eva}<br>
      ({lat:.6f}, {lon:.6f})
    </div>
    """

    folium.CircleMarker(
        location=[lat, lon],
        radius=POINT_RADIUS,
        weight=3,          # klarer Rand
        opacity=1.0,       # Rand voll deckend
        fill=True,
        fill_opacity=1.0,  # Punkt voll deckend
        tooltip=folium.Tooltip(tooltip_html, sticky=True),
        popup=folium.Popup(popup_html, max_width=350),
    ).add_to(fg)

folium.LayerControl().add_to(m)

conn.close()
m

Stationen (Berlin-BBox): 133
